In [31]:
import numpy as np
import pandas as pd

import os

if not os.path.exists("images"):
    os.mkdir("images")

In [32]:
monthly = pd.read_csv('zip9_coded_201909_wh.csv')

In [33]:
monthly.head()

,zip5,zip9_code,bankcard_limit,bankcard_balance,bankcard_trades,bankcard_util,total_revolving_limit,total_revolving_balance,total_revolving_trades,total_revolving_util,...,homeequity5_open,total_homeequity_limit,total_homeequity_balance,total_homeequity_trades,homeequity1_loan_to_value,autoloan_open,studentloan_open,bankcard_open,homeequity_open,mortgage_open
0,48642,19,27890.000000,5856.800000,2.400000,0.453984,38360.700000,11926.000000,4.900000,0.397245,...,0.0,10000.0,6503.0,0.200,65.0,0.400000,0.1000,0.700000,0.2000,0.700000
1,84401,26,10540.384615,597.384615,1.923077,0.295520,12853.769231,1069.153846,3.461538,0.310865,...,0.0,NaN,NaN,0.000,NaN,0.153846,0.0000,0.615385,0.0000,0.230769
2,33306,34,34677.500000,5236.062500,3.062500,0.263185,39038.375000,5763.437500,4.437500,0.250530,...,0.0,100000.0,0.0,0.125,0.0,0.125000,0.3125,0.937500,0.1875,0.562500
3,30635,41,2130.000000,988.400000,0.600000,0.527898,3161.700000,1790.400000,1.300000,0.772320,...,0.0,NaN,NaN,0.000,NaN,0.100000,0.1000,0.300000,0.0000,0.000000
4,33770,42,10335.000000,7250.500000,3.625000,0.758408,17663.375000,11296.375000,7.250000,0.656051,...,0.0,NaN,NaN,0.000,NaN,0.500000,0.1250,0.875000,0.0000,0.125000


In [34]:
zip9_test = pd.read_csv('zip9_demographics_unlabeled_wh_test.csv')

In [35]:
zip9_test.shape

(667418, 4)

In [36]:
zip1 = pd.merge(zip9_test, monthly.loc[:,['zip5', 'zip9_code']], on='zip9_code')

In [37]:
zip1.head()

,zip9_code,age,household_count,person_count,zip5
0,3797056,44.246817,6,12,1001
1,3068313,37.451611,8,21,1001
2,3203991,51.953018,6,7,1001
3,324866,46.804237,5,9,1001
4,2899974,54.523189,5,10,1001


In [38]:
zip9 = zip1.rename(columns={'zip9_code': 'zip9'})

In [39]:
zip9.shape

(667418, 5)

In [40]:
zip9.head()

,zip9,age,household_count,person_count,zip5
0,3797056,44.246817,6,12,1001
1,3068313,37.451611,8,21,1001
2,3203991,51.953018,6,7,1001
3,324866,46.804237,5,9,1001
4,2899974,54.523189,5,10,1001


In [41]:
left = zip9.loc[:, ['zip5', 'zip9']]

In [42]:
# get age first
df = (
    zip9
    .groupby('zip5', as_index=False)
    .agg({'age': 'mean'})
    .rename(columns={'age': 'mean_age'})
    )

df['mean_age'] = (df['mean_age'] - np.mean(df['mean_age']))/np.std(df['mean_age'])

df.shape

(21136, 2)

In [43]:
df.head()

,zip5,mean_age
0,1001,0.127710
1,1002,-0.351784
2,1005,-0.098022
3,1007,0.298482
4,1008,0.433046


In [44]:
df = pd.merge(left, df, on='zip5', how='inner')

In [46]:
# get home value
home_value = pd.read_csv('home-value.csv', encoding='latin1')

In [47]:
home_value = home_value.loc[:, ['RegionName', '2019-10']]

home_value['2019-10'] = (home_value['2019-10'] - np.mean(home_value['2019-10'])) / np.std(home_value['2019-10'])

home_value = home_value.rename(columns={'RegionName': 'zip5', '2019-10': 'median_home_value'})

home_value.head()

,zip5,median_home_value
0,10025,4.608868
1,60657,1.177292
2,10023,4.089417
3,77494,0.521178
4,60614,1.847468


In [48]:
home_value.shape

(30442, 2)

In [49]:
# get income by zipcode
income = pd.read_csv('income.csv')

In [50]:
income = income.loc[:, ['ZIPCODE', 'Avg total income']]

income['standardized_income'] = (income['Avg total income'] - np.mean(income['Avg total income'])) / np.std(income['Avg total income'])

income = income.rename(columns={'ZIPCODE': 'zip5'})

income = income.drop(columns=['Avg total income'])


income.head()

,zip5,standardized_income
0,0,-0.137908
1,35004,-0.144876
2,35005,-0.467090
3,35006,-0.260412
4,35007,-0.096111


In [51]:
income.shape

(27790, 2)

In [52]:
merge = pd.merge(df, home_value, on='zip5', how='left')

In [53]:
merge = pd.merge(merge, income, on='zip5', how='left')

In [54]:
merge.head()

,zip5,zip9,mean_age,median_home_value,standardized_income
0,1001,3797056,0.12771,-0.002525,-0.094343
1,1001,3068313,0.12771,-0.002525,-0.094343
2,1001,3203991,0.12771,-0.002525,-0.094343
3,1001,324866,0.12771,-0.002525,-0.094343
4,1001,2899974,0.12771,-0.002525,-0.094343


In [ ]:
#merge['affordability'] = merge['median_home_value'] / (merge['Avg total income'] * 1000)

In [ ]:
# affordability is a measure of years the average person can afford a median house

In [ ]:
merge.head()

In [55]:
zip9.head()

,zip9,age,household_count,person_count,zip5
0,3797056,44.246817,6,12,1001
1,3068313,37.451611,8,21,1001
2,3203991,51.953018,6,7,1001
3,324866,46.804237,5,9,1001
4,2899974,54.523189,5,10,1001


In [56]:
num_households = (
    zip9
    .groupby('zip5', as_index=False)
    .agg({'household_count': 'sum'})

)

In [ ]:
home_buyers = (
    zip9
    .groupby('zip5', as_index=False)
    .agg({'homebuyers': 'sum'})

)

home_buyers.head()

In [57]:
merge = pd.merge(merge, num_households, on='zip5', how='left')

In [ ]:
merge = pd.merge(merge, home_buyers, on='zip5', how='left')

In [ ]:
import plotly.graph_objects as go
import numpy as np


In [ ]:
fig = go.Figure(data=go.Scatter(
    y = imputed_merge['affordability'],
    x = imputed_merge['homebuyers'],
    mode='markers',
#     marker=dict(
#         size=12,
#         color=merge['household_count'], #set color equal to a variable
#         colorscale='Viridis', # one of plotly colorscales
#         showscale=True
#     )
))

fig.show()

In [ ]:
imputed_merge.corr()

In [ ]:
large_communities = merge.loc[merge['household_count'] < 500]

In [ ]:
fig = go.Figure(data=go.Scatter(
    y = large_communities['Avg total income'],
    x = large_communities['median_home_value'],
    mode='markers',
    marker=dict(
        size=12,
        color=large_communities['household_count'], #set color equal to a variable
        colorscale='Viridis', # one of plotly colorscales
        showscale=True
    )
))

fig.show()

In [ ]:
plt.hist(merge['household_count'])

In [ ]:
merge['household_count'].describe()

In [ ]:
monthly_listings = pd.read_csv('monthly-listings.csv', encoding='latin1')

In [ ]:
# can't use monthly listings since only 12000 zipcodes of data
# # get monthly listings
# monthly_listings = pd.read_csv('monthly-listings.csv', encoding='latin1')

# monthly_listing_cols = ['RegionId', '2019-01', '2019-02', '2019-03',
#                         '2019-04', '2019-05', '2019-06', '2019-07', '2019-08',
#                         '2019-09', '2019-10', '2019-11', '2019-12']

# monthly_listings = monthly_listings.loc[:, monthly_listing_cols]

# monthly_listings['avg_listings'] = (monthly_listings['2019-01'] + monthly_listings['2019-02'] + \
# monthly_listings['2019-03'] + monthly_listings['2019-04'] + monthly_listings['2019-05'] + monthly_listings['2019-06'] +\
# monthly_listings['2019-07'] + monthly_listings['2019-08'] +  monthly_listings['2019-09'] + monthly_listings['2019-10'] +\
# monthly_listings['2019-11'] + monthly_listings['2019-12'])/12

# monthly_listings = monthly_listings.loc[:, ['RegionId', 'avg_listings']]

# monthly_listings['avg_listings'] = (monthly_listings['avg_listings'] - np.mean(monthly_listings['avg_listings'])) / np.std(monthly_listings['avg_listings'])

# monthly_listings.head()

In [ ]:
# can't use new monthly lisitngs since we have 5000
# get new monthly listings
# new_listings = pd.read_csv('new-monthly-listings.csv', encoding='latin1')

# monthly_listing_cols = ['RegionName', '2019-01', '2019-02', '2019-03',
#                         '2019-04', '2019-05', '2019-06', '2019-07', '2019-08',
#                         '2019-09', '2019-10', '2019-11', '2019-12']

# new_listings = new_listings.loc[:, monthly_listing_cols]

# new_listings['avg_new_listings'] = (new_listings['2019-01'] + new_listings['2019-02'] + \
# new_listings['2019-03'] + new_listings['2019-04'] + new_listings['2019-05'] + new_listings['2019-06'] +\
# new_listings['2019-07'] + new_listings['2019-08'] +  new_listings['2019-09'] + new_listings['2019-10'] +\
# new_listings['2019-11'] + new_listings['2019-12'])/12

# new_listings = new_listings.loc[:, ['RegionName', 'avg_new_listings']]

# monthly_listings['avg_listings'] = (monthly_listings['avg_listings'] - np.mean(monthly_listings['avg_listings'])) / np.std(monthly_listings['avg_listings'])

# monthly_listings.head()

In [58]:
merge.isnull().sum()

zip5                      0
zip9                      0
mean_age                  0
median_home_value      3197
standardized_income    1808
household_count           0
dtype: int64

In [59]:
merge.columns

Index(['zip5', 'zip9', 'mean_age', 'median_home_value', 'standardized_income',
       'household_count'],
      dtype='object')

In [ ]:
# impute with mean

In [21]:
from sklearn.impute import SimpleImputer

In [60]:
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')

In [61]:
imp_median.fit(merge)

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='median', verbose=0)

In [62]:
imputed_merge = imp_median.transform(merge)

In [63]:
imputed_merge = pd.DataFrame(imputed_merge, columns=merge.columns)

In [64]:
imputed_merge.isnull().sum()

zip5                   0
zip9                   0
mean_age               0
median_home_value      0
standardized_income    0
household_count        0
dtype: int64

In [ ]:
imputed_merge.corr()

In [ ]:
for i in np.arange(50, 20, -1):

    print(str(imputed_merge[imputed_merge['mean_age'] < i].loc[:, ['homebuyers','mean_age']].corr().iloc[0,1]) + \
          str(" less than " + str(i) + " | ") + str(imputed_merge[imputed_merge['mean_age'] > i].loc[:, ['homebuyers','mean_age']].corr().iloc[0,1]) + \
          str(" greater than " + str(i)))

In [ ]:
plt.hist(imputed_merge['mean_age'])

In [ ]:
imputed_merge.head()

In [ ]:
# As household count increases, the affordability of an area increases (means houses are less affordable)
# There is a strong correlation between level of affordability and house hold count in the area

fig = go.Figure(data=go.Scatter(
    y = merge['homebuyers'],
    x = merge['mean_age'],
    mode='markers',
    marker=dict(
        size=8,
        color=merge['household_count'], #set color equal to a variable
        colorscale='Viridis', # one of plotly colorscales
        showscale=True
    )
))

# Plot labels
fig.update_layout(title='',
                  xaxis_title='Average Age',
                  yaxis_title='Total Homes Bought',
                  annotations=[
                      
                  ],
                 shapes=[
                     go.layout.Shape(
                        type="line",
                        x0=43,
                        y0=0,
                        x1=43,
                        y1=185,
                        line=dict(
                            color="red",
                            width=4,
                            dash="dot",
                            ))])



fig.show()

fig.write_image("images/age.png")


In [ ]:
greater31 = imputed_merge[imputed_merge['mean_age'] >= 45]#.loc[imputed_merge['homebuyers'] !=0]

In [ ]:
merge.head()

In [ ]:
some_slice = merge.loc[:, ['median_home_value', 'Avg total income', 'affordability', 'homebuyers']].corr()

In [ ]:
import seaborn as sns

In [ ]:
ax = sns.heatmap(some_slice, cmap="YlGnBu")

In [ ]:
merge.loc[:, ['median_home_value', 'household_count']].corr().iloc[0,1]

In [ ]:
corrs = []
for i in np.arange(10000):
    sample = merge.sample(len(merge), replace=True)
    corrs.append(sample.loc[:, ['affordability', 'homebuyers']].corr().iloc[0, 1])

In [ ]:
fig = go.Figure(data=[go.Histogram(x=corrs, histnorm='probability')])

fig.update_layout(title='',
                  xaxis_title='correlation',
                  yaxis_title='density',
                 annotations=[
                    
                      ],
                 )
fig.update_xaxes(range=[0.2, 0.26])

fig.show()

In [ ]:
fig = go.Figure(data=go.Scatter(
    y = merge['affordability'],
    x = merge['household_count'],
    mode='markers',
    marker=dict(
        size=12,
#         color='blue',#large_communities['household_count'], #set color equal to a variable
#         colorscale='Viridis', # one of plotly colorscales
#         showscale=True
    )
))

fig.show()

In [ ]:
fig = go.Figure(data=go.Scatter(
    y = greater31['homebuyers'],
    x = greater31['mean_age'],
    mode='markers',
    marker=dict(
        size=12,
#         color='blue',#large_communities['household_count'], #set color equal to a variable
#         colorscale='Viridis', # one of plotly colorscales
#         showscale=True
    )
))

fig.show()

In [ ]:
# the underlying story is that higher amount of households in an area (e.g. large communities) leads to both 
# less affordable houses. People have 

# gaining economic advantage (buy low sell high)
# ideally want to move to a place where there is a large community
# at a circumstance when you need to buy a house (age maturity)

# we didn't take into account necessity (people need to move with work). Drivers should account for majority

In [65]:
imputed_merge.head()

,zip5,zip9,mean_age,median_home_value,standardized_income,household_count
0,1001.0,3797056.0,0.12771,-0.002525,-0.094343,406.0
1,1001.0,3068313.0,0.12771,-0.002525,-0.094343,406.0
2,1001.0,3203991.0,0.12771,-0.002525,-0.094343,406.0
3,1001.0,324866.0,0.12771,-0.002525,-0.094343,406.0
4,1001.0,2899974.0,0.12771,-0.002525,-0.094343,406.0


In [70]:
imputed_merge['zip5'] = imputed_merge['zip5'].apply(int)
imputed_merge['zip9'] = imputed_merge['zip9'].apply(int)
imputed_merge['household_count'] = imputed_merge['household_count'].apply(int)

In [71]:
imputed_merge.head()

,zip5,zip9,mean_age,median_home_value,standardized_income,household_count
0,1001,3797056,0.12771,-0.002525,-0.094343,406
1,1001,3068313,0.12771,-0.002525,-0.094343,406
2,1001,3203991,0.12771,-0.002525,-0.094343,406
3,1001,324866,0.12771,-0.002525,-0.094343,406
4,1001,2899974,0.12771,-0.002525,-0.094343,406


In [ ]:
imputed_merge.drop(columns=['homebuyers'], inplace=True)

In [72]:
imputed_merge.to_csv("test-data.csv", index=False)

In [ ]:
imputed_merge

In [ ]:
df = pd.read